# Exercise 1: End-to-End Synthetic Data Generation Flow

In this exercise, we will build a comprehensive synthetic dataset for a **Service Desk** use case entirely from scratch using NVIDIA Data Designer. This hands-on session covers the full lifecycle of synthetic data generation—from defining the schema to evaluating quality.

### **Learning Objectives**
By the end of this exercise, you will know how to:

1.  **Define Structural Data**: Use deterministic **Samplers** to create structured fields like IDs, scores, names, and categories.
2.  **Generate Realistic Text**: Leverage **LLMs** to generate context-aware ticket descriptions, agent resolutions, and customer feedback.
3.  **Implement Business Logic**: Use **Expression Columns** to derive new data points based on generated values (e.g., flagging low CSAT scores).
4.  **Evaluate Quality**: Set up an **LLM-as-a-Judge** to automatically score the quality (Professionalism, Relevance) of generated resolutions.
5.  **Analyze Results**: Generate, export, and profile the final dataset.

---

### 1. Imports and Setup
Import necessary classes from `data_designer`.

In [ ]:
from data_designer.essentials import (
    DataDesigner,
    ModelConfig,
    InferenceParameters,
    CategorySamplerParams,
    DataDesignerConfigBuilder,
    ExpressionColumnConfig,
    JudgeScoreProfilerConfig,
    LLMJudgeColumnConfig,
    LLMTextColumnConfig,
    PersonFromFakerSamplerParams,
    Score,
    SamplerColumnConfig,
    SamplerType,
    SubcategorySamplerParams,
    UniformSamplerParams,
    UUIDSamplerParams,
)

In [ ]:
from data_designer.config.models import ModelProvider

### 2. Configure Model Provider
Set up the connection to the Local NIM (NVIDIA Inference Microservice).

In [ ]:
local_nim_provider = ModelProvider(
    name="local-nim",
    endpoint="http://localhost:8080/v1",
    provider_type="openai",
    api_key="dummy"
)

In [ ]:
data_designer = DataDesigner(model_providers=[local_nim_provider])

In [ ]:
MODEL_ID = "nvidia/nvidia-nemotron-nano-9b-v2" 
MODEL_PROVIDER = "local-nim"
MODEL_ALIAS = "local-model"
SYSTEM_PROMPT = "/no_think"

### 3. Define Model Configuration
Specify the model to be used for generation (e.g., `nvidia-nemotron-nano-9b-v2`).

In [ ]:
model_configs = [
    ModelConfig(
        alias=MODEL_ALIAS,
        model=MODEL_ID,
        provider=MODEL_PROVIDER,
        inference_parameters=InferenceParameters(
            temperature=0.5,
            top_p=1.0,
            max_tokens=1024
        )
    )
]

### 4. Initialize Config Builder
Start building the data generation configuration.

In [ ]:
config_builder = DataDesignerConfigBuilder(model_configs=model_configs)

### 5. Define Base Columns (Samplers)
We start by defining the structural columns of our dataset using deterministic samplers.
- `ticket_id`: Unique identifier.
- `priority_score`: Random score between 1-10.
- `employee`: Fake persona.
- `department`: Categorical selection.
- `issue_type`: Dependent on department (Subcategory).

In [ ]:
config_builder.add_column(
    SamplerColumnConfig(
        name="ticket_id",
        sampler_type=SamplerType.UUID,
        params=UUIDSamplerParams(prefix="INC-", short_form=True)
    )
)

In [ ]:
config_builder.add_column(
    SamplerColumnConfig(
        name="priority_score",
        sampler_type=SamplerType.UNIFORM,
        params=UniformSamplerParams(
            low=1.0,
            high=10.0,
            decimal_places=2  # Round to 2 decimal places
        )
    )
)

In [ ]:
config_builder.add_column(
    SamplerColumnConfig(
        name="employee",
        sampler_type=SamplerType.PERSON_FROM_FAKER,
        params=PersonFromFakerSamplerParams(
            locale="en_US",
            age_range=[22, 65]
        )
    )
)

In [ ]:
config_builder.add_column(
    SamplerColumnConfig(
        name="department",
        sampler_type=SamplerType.CATEGORY,
        params=CategorySamplerParams(
            values=["IT Support", "HR", "Facilities"],
            weights=[0.5, 0.3, 0.2]  # IT Support is most common
        )
    )
)

In [ ]:
config_builder.add_column(
    SamplerColumnConfig(
        name="issue_type",
        sampler_type=SamplerType.SUBCATEGORY,
        params=SubcategorySamplerParams(
            category="department",  # Links to the column above
            values={
                "IT Support": ["Login Failure", "Hardware Malfunction", "Software Installation", "VPN Issue"],
                "HR": ["Benefits Inquiry", "Payroll Discrepancy", "Conflict Resolution"],
                "Facilities": ["Desk Repair", "Temperature Control", "Access Badge"],
            },
        ),
    )
)

In [ ]:
config_builder.validate()

### 6. Generate Text Content (LLM Columns)
Now we use the LLM to generate realistic text content based on the sampled values.
- `ticket_description`: Generated based on employee, department, and issue type.

In [ ]:
PROMPT_TEMPLATE = """
You are {{ employee.first_name }} {{ employee.last_name }}, an employee in the {{ department }} department.
You are submitting a helpdesk ticket regarding: {{ issue_type }}.
Your priority score is {{ priority_score }}/10.

Write a short description of your issue.
If priority is high (>8), sound urgent and frustrated.
If priority is low (<4), sound casual.
Include specific details relevant to {{ issue_type }}.

Output ONLY the ticket description.
"""

In [ ]:
config_builder.add_column(
    LLMTextColumnConfig(
        name="ticket_description",
        system_prompt=SYSTEM_PROMPT,
        model_alias=MODEL_ALIAS,
        prompt=PROMPT_TEMPLATE,
    )
)

In [ ]:
config_builder.validate()

- `agent_resolution`: Generated based on the ticket description.

In [ ]:
RESOLUTION_PROMPT = """
You are an expert Helpdesk Agent.
Ticket: {{ ticket_description }}
Issue Type: {{ issue_type }}

Write a brief, professional resolution note explaining how you solved this issue.
Be specific to the technical details mentioned in the ticket.
"""

In [ ]:
config_builder.add_column(
    LLMTextColumnConfig(
        name="agent_resolution",
        system_prompt=SYSTEM_PROMPT,
        model_alias=MODEL_ALIAS,
        prompt=RESOLUTION_PROMPT,
    )
)

- `csat_score`: Simulated customer satisfaction score.

In [ ]:
CSAT_PROMPT = """
You are {{ employee.first_name }}.
You submitted a ticket: "{{ ticket_description }}"
The agent resolved it with: "{{ agent_resolution }}"

Rate your satisfaction with this resolution on a scale of 1 to 5 (5 being best).
Consider:
- Did they address your specific details?
- Was the tone appropriate?

Output ONLY the number (1, 2, 3, 4, or 5).
"""

In [ ]:
config_builder.add_column(
    LLMTextColumnConfig(
        name="csat_score",
        system_prompt=SYSTEM_PROMPT,
        model_alias=MODEL_ALIAS,
        prompt=CSAT_PROMPT
    )
)

### 7. Derived Columns (Expressions)
Create columns based on logical expressions derived from other columns.
- `follow_up_required`: Logic to flag low CSAT scores.

In [ ]:
config_builder.add_column(
    ExpressionColumnConfig(
        name="follow_up_required",
        expr="{% if csat_score | int < 3 %}YES{% else %}NO{% endif %}",
    )
)

In [ ]:
config_builder.validate()

### 8. Quality Evaluation (LLM Judge)
Add an LLM-as-a-Judge to evaluate the quality of the generated agent resolutions.

In [ ]:
config_builder.add_column(
    LLMJudgeColumnConfig(
        name="quality_evaluation",
        model_alias=MODEL_ALIAS,
        prompt="Evaluate the quality of the following agent resolution:\n\nResolution: {{ agent_resolution }}\nTicket: {{ ticket_description }}",
        scores=[
            Score(
                name="Professionalism",
                description="Evaluate the professional tone of the response.",
                options={1: "Rude", 3: "Neutral", 5: "Highly Professional"},
            ),
            Score(
                name="Relevance",
                description="Does the resolution directly address the issue described in the ticket?",
                options={1: "Irrelevant", 3: "Somewhat Relevant", 5: "Highly Relevant"},
            ),
        ],
    )
)

In [ ]:
config_builder.add_profiler(
    JudgeScoreProfilerConfig(model_alias=MODEL_ALIAS)
)

In [ ]:
config_builder.validate()

### 9. Preview Generation
Generate a small sample (5 records) to verify the configuration and quality before running the full job.

In [ ]:
preview = data_designer.preview(config_builder, num_records=5)

In [ ]:
preview.display_sample_record()

In [ ]:
preview.dataset.head(5)

In [ ]:
preview.analysis.to_report()

In [ ]:
preview.dataset

### 10. Full Dataset Generation
Generate the complete dataset (10 records for this demo) and save it.

In [ ]:
result = data_designer.create(config_builder, num_records=10, dataset_name='ServiceDesk-SDG')

In [ ]:
dataset = result.load_dataset()

In [ ]:
dataset.head()

### 11. Analysis
Review the generated data statistics and judge scores.

In [ ]:
analysis = result.load_analysis()

In [ ]:
analysis.to_report()